## A CSV to aggregate user Data

In [1]:
import pandas as pd
import re
import os
import json

In [2]:
df = pd.read_csv("../../data/unique_artists_6.csv", delimiter=";")

In [7]:
folder = "artist_jsons"

In [8]:
files = os.listdir(folder)

In [9]:
out = list()

In [10]:
jsons = list()

We Unpack artist data from Jsons

In [11]:
# function to unpack jsons
def unpack_json(dic):
    lifespan = dic["life-span"]
    lifespan["life_end"] = lifespan.pop("end")
    lifespan["life_ended"] = lifespan.pop("ended")
    lifespan["life_begin"] = lifespan.pop("begin")
    dic.pop("life-span")
    dic.update(lifespan)
    for field in ["begin_area", "area", "end_area"]:
        if field in dic:
            if dic[field]:
                new_field = dic[field]
                dic.pop(field)
                dic[field] = new_field.pop("name")
    for field in ["genres", "tags"]:
        if field in dic:
            if dic[field]:
                array = []
                for i in dic[field]:
                    array.append(i.pop("name"))
                    dic[field] = array
            else:
                dic[field] = None

In [12]:
rows_list = []
for row in df["artists_names"]:
    clean_filename = re.sub(r"[><:\"?*|\\/]", "", f"{row}.json")
    path = os.path.join(folder, clean_filename)
    base_info = {
        "artists_names":row
        }
    if os.path.isfile(path):
        with open(path, "r") as js:
            json_data = json.load(js)
            unpack_json(json_data)
            base_info.update({"json_filename" : clean_filename})
            base_info.update(json_data)
            rows_list.append(base_info)
    else:
        rows_list.append(base_info)

out = pd.DataFrame(rows_list)       

In [13]:
out.drop(["works", "releases", "release-groups", "aliases", "type-id", "recordings", "gender-id", "begin-area", "end-area",  "rating", "isnis", "ipis", "relations"], axis=1).to_csv("artistinfo_extended2.csv", sep=";", index=False)

In [14]:
out['json_filename'].isnull().sum()

33

In [15]:
out[out['json_filename'].isnull()]["artists_names"].to_csv("nas.csv", sep=";", index=False)